#  Train a GPT-2 Text-Generating Model
Retrain an advanced text generating neural network on any text dataset 

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Thu Nov  7 16:04:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

In [3]:
# MODEL_NAME = '774M' # is not trainable
MODEL_NAME = '355M'
gpt2.download_gpt2(model_name=MODEL_NAME)

Fetching checkpoint: 1.05Mit [00:00, 359Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 69.9Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 497Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:17, 82.2Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 214Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 90.8Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 123Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


we do the same as in [1-horoscopes-preparation.ipynb](https://drive.google.com/file/d/1qeLw2Xm0OEZPH91gv6Zw08z1gydooslb/view?usp=sharing)

In [5]:
!wget 'https://raw.githubusercontent.com/ischender/ML_workshop/master/data/horoscopes.csv'

--2019-11-07 16:04:27--  https://raw.githubusercontent.com/ischender/ML_workshop/master/data/horoscopes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3336362 (3.2M) [text/plain]
Saving to: ‘horoscopes.csv.2’

horoscopes.csv.2    100%[===================>]   3.18M  --.-KB/s    in 0.1s    

2019-11-07 16:04:29 (31.0 MB/s) - ‘horoscopes.csv.2’ saved [3336362/3336362]



In [0]:
import pandas as pd

horoscopes = pd.read_csv('horoscopes.csv', sep='|', index_col=0, header=None, names=['horoscope', 'date', 'sign'])
horoscopes['s'] = horoscopes['sign'].apply(lambda x: x + ': ')
horoscopes[['s', 'horoscope']].to_csv('horoscopes.txt', header=None, index=None, sep=' ', mode='a')
lines = []
with open('horoscopes.txt') as fp:
    for line in fp:
        lines.append(line.replace('"', ''))
with open('horoscopes.txt', 'w+') as fp:
    fp.writelines(lines)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
file_name = "horoscopes.txt"

In [0]:
# gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [9]:
%%time 

sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=MODEL_NAME,
              steps=1000,
              restore_from='fresh',
              run_name='horoscopes',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.14s/it]


dataset has 2205849 tokens
Training...
[10 | 42.95] loss=2.29 avg=2.29
[20 | 75.14] loss=2.26 avg=2.28
[30 | 107.26] loss=2.25 avg=2.27
[40 | 139.35] loss=2.15 avg=2.24
[50 | 171.44] loss=2.11 avg=2.21
[60 | 203.46] loss=2.00 avg=2.18
[70 | 235.42] loss=1.80 avg=2.12
[80 | 267.43] loss=1.61 avg=2.05
[90 | 299.38] loss=2.05 avg=2.05
[100 | 331.40] loss=1.92 avg=2.04
[110 | 363.44] loss=2.06 avg=2.04
[120 | 395.52] loss=1.97 avg=2.04
[130 | 427.56] loss=1.63 avg=2.00
[140 | 459.56] loss=1.98 avg=2.00
[150 | 491.61] loss=1.70 avg=1.98
[160 | 523.71] loss=1.96 avg=1.98
[170 | 555.71] loss=1.85 avg=1.97
[180 | 587.74] loss=1.86 avg=1.96
[190 | 619.80] loss=1.87 avg=1.96
[200 | 651.85] loss=1.86 avg=1.95
======== SAMPLE 1 ========
 Wednesday. But now that you have that out of your system you will need to reawaken your senses. Your only choice now is to make a better one.
virgo:  Over the next 24 hours the situation will be made worse by the day, so stop worrying about being right and make th

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='horoscopes')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='horoscopes')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='horoscopes')

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [12]:
gpt2.generate(sess, run_name='horoscopes', prefix='Shauna:', truncate='\n')


Shauna:  The Jupiter-Saturn link in your sign makes you quite impulsive, and if you stray too far you could miss out. At some stage today you will realize that you have taken a negative view of your life and people will come to you for help.


In [0]:
import requests
import re

bible_url = 'http://www.gutenberg.org/ebooks/8294.txt.utf-8'
r = requests.get(bible_url)
bible = r.text
bible_pieces = bible.split('Book ')
bible_pieces_selected = bible_pieces[4:70]

def extract_text(book):
  chapter_list = []
  lines = []
  line = []
  current_chapter = False
  bible_lines = book.split('\n')
  chapter_list.append(bible_lines[0].strip())
  for l in bible_lines[1:1000]:
    r = re.findall(r'^([0-9]{3}):([0-9]{3})(.*)', l)
    # print(r)
    if r:
      chapter_num = r[0][0]
      if current_chapter != chapter_num:
        chapter = ' '.join(lines)
        if '{' in chapter:
          rest = re.findall(r'(.*)\{.*\}(.*)', chapter)
          if rest:
            # print(rest)
            chapter = ''.join(rest[0])
        chapter_list.append(chapter)
        lines = []
        current_chapter = chapter_num
      line = r[0][-1].strip()
    else:
      line = l.strip()
    lines.append(line)
  chapter_list
  chapter_text = '\n\n'.join(chapter_list)
  return chapter_text

bible_pieces_cleaned = [extract_text(chap) for chap in bible_pieces_selected]

bible_final = '\n\n'.join(bible_pieces_cleaned)

In [14]:
bible_final[:100]

'01 Genesis\n\n\n\nIn the beginning God created the heavens and the earth. Now the earth was formless and'

In [15]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=bible_final,
              model_name=MODEL_NAME,
              steps=1000,
              restore_from='fresh',
              run_name='bible',
              print_every=10,
              sample_every=200,
              save_every=500
              )

ValueError: ignored

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='bible')

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M "large" model.

In [0]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [0]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [0]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

In [0]:
!kill -9 -1